<a href="https://colab.research.google.com/github/kangmg/image2music/blob/main/backend/modeling/LLM_%EA%B8%B0%EB%B0%98_%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Free tier

- claude : https://console.anthropic.com/ # 14일동안 $5 제공

- mistral : https://console.mistral.ai/ # 14일동안 $5 제공


- Solar : https://www.upstage.ai/?utm_term=Demo&utm_content=%2Fhome#demo # $60 제공


pay-as-you-go

- Gemini pro : google cloud $300 제공

- GPT4 : 유료

In [6]:
#@title package install

from IPython.display import clear_output

!pip install -q openai
!pip install -U -q google-generativeai
#!pip install -q llmx

clear_output()

!echo "Dependencies are successfully installed!"

Dependencies are successfully installed!


In [3]:
#@title API setting

# API api key 획득
from google.colab import userdata

OPENAI_API = userdata.get('OPENAI')
GEMINI_API = userdata.get('GEMINI')
SOLAR_API = userdata.get('SOLAR')  # 무료 리딤 쿠폰 WELCOME_2024 / $60
LLAMA_API = userdata.get('LLAMA')


In [7]:
#@title LLM_classifier

from openai import OpenAI # 1.33.0
import google.generativeai as genai # 0.6.0
from time import sleep


def prompt_to_gemini(gpt_roll):
  """
  Convert GPT style role prompt to Gemini style prompt

  Ref. https://www.googlecloudcommunity.com/gc/AI-ML/Implementing-System-Prompts-in-Gemini-Pro-for-Chatbot-Creation/m-p/715501
  """
  messages_gemini = []
  system_promt = ''
  # parsing the gpt-style role prompt
  for role in gpt_roll:
      if role['role'] == 'system':
          system_promt = role['content']
      elif role['role'] == 'user':
          messages_gemini.append({'role': 'user', 'parts': [role['content']]})
  # messages to prompt
  if system_promt:
      messages_gemini[0]['parts'].insert(0, f"*{system_promt}*")

  return messages_gemini


def GPT4o(emotion_tag, api_key=OPENAI_API):
  """
  GPT4o emotion classifier
  """
  # gpt client
  client = OpenAI(
    api_key=OPENAI_API
  )
  # setup model
  stream = client.chat.completions.create(
    model="gpt-4o",
    messages=[
      {
          "role": "system",
          "content": "You're a Classifier that takes mood related with music as words input and returns the closest mood classification among 13 categories. // \n\n Mood classifications list = ['A. Amusing', 'B. Annoying', 'C. Anxious, tense', 'D. Beautiful', 'E. Calm, Relaxing, Serene', 'F. Dreamy', 'G. Energizing, Pump-up', 'H. Erotic, Desirous', 'I. Indignant, Defiant', 'J. Joyful, Cheerful', 'K. Sad, Depressing', 'L. Scary, Fearful', 'M. Triumphant, Heroic']. // \n\n Return the closest Mood from the list."
      },
      {
          "role": "user",
          "content": f"'Which class does this mood belong to? Just tell me the class.' // MOOD = {emotion_tag}"
      }
    ],stream=False
  )
  # generate response
  response = stream.choices[0].message.content
  return response


def Solar(emotion_tag, api_key=SOLAR_API):
  """
  Solar emotion classifier
  """
  # solar client
  client = OpenAI(
    api_key=SOLAR_API,
    base_url="https://api.upstage.ai/v1/solar"
  )
  # setup model
  stream = client.chat.completions.create(
    model="solar-1-mini-chat",
    messages=[
      {
        "role": "system",
        "content": "You're a Classifier that takes mood related with music as words input and returns the closest mood classification among 13 categories. // \n\n Mood classifications list = ['A. Amusing', 'B. Annoying', 'C. Anxious, tense', 'D. Beautiful', 'E. Calm, Relaxing, Serene', 'F. Dreamy', 'G. Energizing, Pump-up', 'H. Erotic, Desirous', 'I. Indignant, Defiant', 'J. Joyful, Cheerful', 'K. Sad, Depressing', 'L. Scary, Fearful', 'M. Triumphant, Heroic']. // \n\n Return the closest Mood from the list."
      },
      {
        "role": "user",
        "content": f"'Which class does this mood belong to? Just tell me the class.' // MOOD = {emotion_tag}"
      }
    ],
    stream=False
  )
  # generate response
  response = stream.choices[0].message.content
  return response

def LLAMA(emotion_tag, api_key=LLAMA_API):
  """
  llama emotion classifier
  """
  # llama client
  client = OpenAI(
    api_key = LLAMA_API, # llama api key
    base_url = "http://sionic.chat:8001/v1",
  )
  stream = client.chat.completions.create(
    model="xionic-ko-llama-3-70b",
    messages=[
      {
        "role": "system",
        "content": "You're a Classifier that takes mood related with music as words input and returns the closest mood classification among 13 categories. // \n\n Mood classifications list = ['A. Amusing', 'B. Annoying', 'C. Anxious, tense', 'D. Beautiful', 'E. Calm, Relaxing, Serene', 'F. Dreamy', 'G. Energizing, Pump-up', 'H. Erotic, Desirous', 'I. Indignant, Defiant', 'J. Joyful, Cheerful', 'K. Sad, Depressing', 'L. Scary, Fearful', 'M. Triumphant, Heroic']. // \n\n Return the closest Mood from the list. \n IMPORTANT : DO NOT include any description other than the class names in the list."
      },
      {
        "role": "user",
        "content": f"'Which class does this mood belong to? Just tell me the class.' // MOOD = {emotion_tag}"
      }
    ],
    stream=False
  )
  # generate response
  response = stream.choices[0].message.content
  return response


def Gemini_15_pro(emotion_tag, api_key=GEMINI_API):
  """
  Gemini emotion classifier
  """
  # Gemini role prompt
  gpt_style_prompt=[
      {
        "role": "system",
        "content": "You're a Classifier that takes mood related with music as words input and returns the closest mood classification among 13 categories. // \n\n Mood classifications list = ['A. Amusing', 'B. Annoying', 'C. Anxious, tense', 'D. Beautiful', 'E. Calm, Relaxing, Serene', 'F. Dreamy', 'G. Energizing, Pump-up', 'H. Erotic, Desirous', 'I. Indignant, Defiant', 'J. Joyful, Cheerful', 'K. Sad, Depressing', 'L. Scary, Fearful', 'M. Triumphant, Heroic']. // \n\n Return the closest Mood from the list. \n IMPORTANT : DO NOT include any description other than the class names in the list."
      },
      {
        "role": "user",
        "content": f"'Which class does this mood belong to? Just tell me the class.' // MOOD = {emotion_tag}"
        }
      ]
  message = prompt_to_gemini(gpt_style_prompt)
  # setup model and configure api
  genai.configure(api_key=GEMINI_API)
  model = genai.GenerativeModel(model_name='gemini-1.5-pro')
  #model = genai.GenerativeModel(model_name='gemini-1.5-flash')
  # generate response
  res = model.generate_content(message)
  sleep(2)
  try:
    response = res.text
  except:
    response = ""
    print(f"MISSING VALUE WARNING : {emotion_tag}")

  # remove \n
  if response.endswith('\n'):
    response = response.rstrip("\n").strip()
  return response

***분류 테스트***

In [18]:
Solar("Driving")

'C. Anxious, tense'

In [19]:
LLAMA("Driving")

'G.'

In [20]:
Gemini_15_pro("Driving")

'G. Energizing, Pump-up'

In [21]:
GPT4o("Driving")

'G. Energizing, Pump-up'

In [22]:
LLAMA("Driving")

'G.'

***Table로 만들기***

In [ ]:
#@title 감정 리스트 정의

MOOD_TAGS_LIST = [
    "Light",
    "Sentimental",
    "Intense",
    "Strong",
    "Uplifting",
    "Aggresive",
    "Sensual",
    "Melancholy",
    "Dramatic",
    "Positive",
    "Dreamy",
    "Romantic",
    "Sweet",
    "Warm",
    "Scary",
    "Bright",
    "Complex",
    "Smooth",
    "Atmospheric",
    "Fierce",
    "Lovely",
    "Lyrical",
    "Sexy",
    "Bittersweet",
    "Sad",
    "Exciting",
    "Mysterious",
    "Serious",
    "Lonely",
    "Gloomy",
    "Epic",
    "Confident",
    "Calm",
    "Fun",
    "Gentle",
    "Energetic",
    "Earnest",
    "Driving",
    "Cold",
    "Intimate",
    "Cheerful",
    "Relaxed",
    "Happy",
    "Nostalgic",
    "Angry"
]

In [ ]:
from tqdm.auto import tqdm
from time import sleep
import pandas as pd

# blank df 정의
MOOD_TABLE = pd.DataFrame()

gpt_out = list()
gemini_out = list()
solar_out = list()
llama_out = list()

for MOOD in tqdm(MOOD_TAGS_LIST):
  # API 호출하는 동안 잠시 대기
  sleep(1)
  gpt_out.append(GPT4o(MOOD))
  gemini_out.append(Gemini_15_pro(MOOD))
  solar_out.append(Solar(MOOD))
  llama_out.append(LLAMA(MOOD))

MOOD_TABLE.index = MOOD_TAGS_LIST
MOOD_TABLE['GPT4o'] = gpt_out
MOOD_TABLE['Gemini_flash'] = gemini_out
MOOD_TABLE['Solar'] = solar_out
MOOD_TABLE['LLAMA'] = llama_out

  0%|          | 0/45 [00:00<?, ?it/s]

MISSING VALUE WARNING : Sensual
MISSING VALUE WARNING : Romantic
MISSING VALUE WARNING : Sexy
MISSING VALUE WARNING : Intimate


In [ ]:
# trial 4
MOOD_TABLE

,GPT4o,Gemini_flash,Solar,LLAMA
Light,D. Beautiful,Please provide me with the mood words you want...,F. Dreamy,E.
Sentimental,'D. Beautiful',"K. Sad, Depressing","E. Calm, Relaxing, Serene",D. Beautiful
Intense,"'C. Anxious, tense'","G. Energizing, Pump-up","G. Energizing, Pump-up","G. Energizing, Pump-up"
Strong,The closest mood classification to 'Strong' fr...,Please provide the mood-related word(s) you'd ...,"C. Anxious, tense","G. Energizing, Pump-up"
Uplifting,"'J. Joyful, Cheerful'","J. Joyful, Cheerful","G. Energizing, Pump-up","J. Joyful, Cheerful"
Aggresive,"'I. Indignant, Defiant'","I. Indignant, Defiant","I. Indignant, Defiant",I
Sensual,"H. Erotic, Desirous",NaN,B. Annoying,"H. Erotic, Desirous"
Melancholy,"The closest mood classification to ""Melancholy...","K. Sad, Depressing","K. Sad, Depressing","K. Sad, Depressing"
Dramatic,"'M. Triumphant, Heroic'","Okay, I understand. Please provide the mood w...","G. Energizing, Pump-up","M. Triumphant, Heroic"
Positive,"'J. Joyful, Cheerful'",Please provide me with the mood words you want...,"J. Joyful, Cheerful","J. Joyful, Cheerful"


In [ ]:
MOOD_TABLE

,GPT4o,Gemini_flash,Solar,LLAMA
Light,D. Beautiful,Please provide me with the mood words you want...,F. Dreamy,E.
Sentimental,'D. Beautiful',"K. Sad, Depressing","E. Calm, Relaxing, Serene",D. Beautiful
Intense,"'C. Anxious, tense'","G. Energizing, Pump-up","G. Energizing, Pump-up","G. Energizing, Pump-up"
Strong,The closest mood classification to 'Strong' fr...,Please provide the mood-related word(s) you'd ...,"C. Anxious, tense","G. Energizing, Pump-up"
Uplifting,"'J. Joyful, Cheerful'","J. Joyful, Cheerful","G. Energizing, Pump-up","J. Joyful, Cheerful"
Aggresive,"'I. Indignant, Defiant'","I. Indignant, Defiant","I. Indignant, Defiant",I
Sensual,"H. Erotic, Desirous",NaN,B. Annoying,"H. Erotic, Desirous"
Melancholy,"The closest mood classification to ""Melancholy...","K. Sad, Depressing","K. Sad, Depressing","K. Sad, Depressing"
Dramatic,"'M. Triumphant, Heroic'","Okay, I understand. Please provide the mood w...","G. Energizing, Pump-up","M. Triumphant, Heroic"
Positive,"'J. Joyful, Cheerful'",Please provide me with the mood words you want...,"J. Joyful, Cheerful","J. Joyful, Cheerful"


### 태영님 감정 데이터셋 분류

In [20]:
#@title MOOD LIST 정의
MOOD_LIST = list(mood for mood in {'admiration': 0,
 'amusement': 0,
 'anger': 0,
 'annoyance': 0,
 'approval': 0,
 'caring': 0,
 'confusion': 0,
 'curiosity': 0,
 'desire': 0,
 'disappointment': 0,
 'disapproval': 0,
 'disgust': 0,
 'embarrassment': 0,
 'excitement': 0,
 'fear': 0,
 'gratitude': 0,
 'grief': 0,
 'joy': 0,
 'love': 0,
 'nervousness': 0,
 'optimism': 0,
 'pride': 0,
 'realization': 0,
 'relief': 0,
 'remorse': 0,
 'sadness': 1,
 'surprise': 0,
 'neutral': 0}.keys())

In [34]:
from tqdm.auto import tqdm
from time import sleep
import pandas as pd

# blank df 정의
MOOD_TABLE_2 = pd.DataFrame()

gpt_out = list()
gemini_out = list()
solar_out = list()
llama_out = list()

for MOOD in tqdm(MOOD_LIST):
  # API 호출하는 동안 잠시 대기
  sleep(1)
  gpt_out.append(GPT4o(MOOD))
  gemini_out.append(Gemini_15_pro(MOOD))
  solar_out.append(Solar(MOOD))
  llama_out.append(LLAMA(MOOD))

MOOD_TABLE_2.index = MOOD_LIST
MOOD_TABLE_2['GPT4o'] = gpt_out
MOOD_TABLE_2['Gemini_1.5pro'] = gemini_out
MOOD_TABLE_2['Solar'] = solar_out
MOOD_TABLE_2['LLAMA'] = llama_out

  0%|          | 0/28 [00:00<?, ?it/s]

MISSING VALUE WARNING : desire


In [35]:
MOOD_TABLE_2.to_csv('MOOD_TABLE_2.csv')
MOOD_TABLE_2

,GPT4o,Gemini_1.5pro,Solar,LLAMA
admiration,D. Beautiful,D. Beautiful,A. Amusing,D.
amusement,A. Amusing,A. Amusing,A. Amusing,A
anger,"I. Indignant, Defiant","I. Indignant, Defiant","I. Indignant, Defiant",I
annoyance,B. Annoying,B. Annoying,B. Annoying,B
approval,"J. Joyful, Cheerful","J. Joyful, Cheerful","C. Anxious, tense","J. Joyful, Cheerful"
caring,"E. Calm, Relaxing, Serene","E. Calm, Relaxing, Serene",B. Annoying,E.
confusion,"'C. Anxious, tense'","C. Anxious, tense",B. Confused,C.
curiosity,"C. Anxious, tense",F. Dreamy,A. Amusing,F.
desire,"H. Erotic, Desirous",,H.,H.
disappointment,"K. Sad, Depressing","K. Sad, Depressing",A. Amusing,K


### trial 1 - 3

In [ ]:
# trial 3
MOOD_TABLE.to_csv('MOOD_TABLE.csv')
MOOD_TABLE

,GPT4o,Gemini_flash,Solar
Light,D. Beautiful,Please provide me with the mood words you want...,F. Dreamy
Sentimental,'D. Beautiful',"K. Sad, Depressing","E. Calm, Relaxing, Serene"
Intense,"'C. Anxious, tense'","G. Energizing, Pump-up","G. Energizing, Pump-up"
Strong,The closest mood classification to 'Strong' fr...,Please provide the mood-related word(s) you'd ...,"C. Anxious, tense"
Uplifting,"'J. Joyful, Cheerful'","J. Joyful, Cheerful","G. Energizing, Pump-up"
Aggresive,"'I. Indignant, Defiant'","I. Indignant, Defiant","I. Indignant, Defiant"
Sensual,"H. Erotic, Desirous",,B. Annoying
Melancholy,"The closest mood classification to ""Melancholy...","K. Sad, Depressing","K. Sad, Depressing"
Dramatic,"'M. Triumphant, Heroic'","Okay, I understand. Please provide the mood w...","G. Energizing, Pump-up"
Positive,"'J. Joyful, Cheerful'",Please provide me with the mood words you want...,"J. Joyful, Cheerful"


In [ ]:
# trial 2
MOOD_TABLE.to_csv('MOOD_TABLE.csv')
MOOD_TABLE

,GPT4o,Gemini_flash,Solar
Light,D. Beautiful,Please provide the emotion words you want me t...,D. Beautiful
Sentimental,F. Dreamy,"J. Joyful, Cheerful",D. Beautiful
Intense,"I. Indignant, Defiant","G. Energizing, Pump-up","G. Energizing, Pump-up"
Strong,"Closest emotion classification: G. Energizing,...","Okay, I'm ready to classify emotions. Please ...","G. Energizing, Pump-up"
Uplifting,"J. Joyful, Cheerful","J. Joyful, Cheerful","G. Energizing, Pump-up"
Aggresive,"I. Indignant, Defiant","I. Indignant, Defiant",B. Annoying
Sensual,"H. Erotic, Desirous",,"H. Erotic, Desirous"
Melancholy,"K. Sad, Depressing","K. Sad, Depressing","K. Sad, Depressing"
Dramatic,"'M. Triumphant, Heroic'","M. Triumphant, Heroic",B. Annoying
Positive,"J. Joyful, Cheerful",Please provide the emotion word you'd like to ...,"J. Joyful, Cheerful"


In [ ]:
# trial 1
MOOD_TABLE

,GPT4o,Solar
Light,D. Beautiful,D. Beautiful
Sentimental,F. Dreamy,D. Beautiful
Intense,"C. Anxious, tense","G. Energizing, Pump-up"
Strong,"M. Triumphant, Heroic","G. Energizing, Pump-up"
Uplifting,"J. Joyful, Cheerful","G. Energizing, Pump-up"
Aggresive,"I. Indignant, Defiant",B. Annoying
Sensual,"H. Erotic, Desirous","H. Erotic, Desirous"
Melancholy,"K. Sad, Depressing","K. Sad, Depressing"
Dramatic,"M. Triumphant, Heroic",A. Amusing
Positive,"J. Joyful, Cheerful","J. Joyful, Cheerful"


Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.
